In [9]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2585/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [8]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [7]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [14]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
7,model_8_9_7,0.999502,0.795189,0.998506,0.999148,0.998875,0.000248,0.121584,0.000555,0.000430,0.000492,0.005005,0.015742,1.000070,0.015815,3534.605625,9487.932441,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
10,model_8_9_6,0.999475,0.795002,0.998629,0.999188,0.998951,0.000261,0.121695,0.000509,0.000409,0.000459,0.005090,0.016156,1.000074,0.016231,3534.501848,9487.828663,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
14,model_8_9_5,0.999436,0.794746,0.998747,0.999225,0.999022,0.000280,0.121847,0.000466,0.000390,0.000428,0.005187,0.016742,1.000079,0.016819,3534.359457,9487.686272,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
20,model_8_9_4,0.999382,0.794442,0.998856,0.999257,0.999087,0.000307,0.122028,0.000425,0.000374,0.000400,0.005284,0.017529,1.000087,0.017610,3534.175695,9487.502511,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
21,model_8_9_3,0.999308,0.794087,0.998951,0.999281,0.999141,0.000344,0.122239,0.000390,0.000362,0.000376,0.005393,0.018555,1.000097,0.018641,3533.948064,9487.274880,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,model_35_2_6,0.998105,0.807630,0.998245,0.998049,0.998166,0.000942,0.114199,0.000962,0.001091,0.001026,0.012090,0.030699,1.000665,0.030840,1683.934166,4509.987543,"Hidden Size=[33, 21], regularizer=0.0, learnin..."
440,model_35_4_8,0.998100,0.807493,0.998945,0.999353,0.999163,0.000945,0.114281,0.000549,0.000364,0.000457,0.012761,0.030738,1.000667,0.030880,1683.928985,4509.982362,"Hidden Size=[33, 21], regularizer=0.0, learnin..."
447,model_35_2_5,0.998095,0.807536,0.998326,0.998055,0.998209,0.000947,0.114255,0.000917,0.001088,0.001002,0.012086,0.030777,1.000669,0.030919,1683.923938,4509.977315,"Hidden Size=[33, 21], regularizer=0.0, learnin..."
464,model_35_2_4,0.998076,0.807157,0.998361,0.998060,0.998228,0.000957,0.114480,0.000898,0.001085,0.000991,0.012102,0.030933,1.000676,0.031076,1683.903723,4509.957101,"Hidden Size=[33, 21], regularizer=0.0, learnin..."
